original: https://www.guruguru.science/competitions/25/discussions/8b97734b-1f76-4075-b1af-5d227d6b70e8/ (@yururoi)


In [1]:
!rm -r /kaggle/working/*
%cd /kaggle/working

/kaggle/working


In [2]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [3]:
import yaml
from penguinml.utils.logger import get_logger, init_logger
from penguinml.utils.set_seed import seed_base

MODEL_NAME = "lightgbm"
CFG = yaml.safe_load(open(os.path.join(PACKAGE_DIR, "config.yaml"), "r"))
print(CFG[MODEL_NAME]["execution"]["exp_id"])
CFG["output_dir"] = f"/kaggle/output/{CFG[MODEL_NAME]['execution']['exp_id']}"
!rm -r {CFG["output_dir"]}
os.makedirs(CFG["output_dir"], exist_ok=True)

init_logger(f"{ CFG[MODEL_NAME]['execution']['exp_id']}.log")
logger = get_logger("main")
seed_base(CFG[MODEL_NAME]["execution"]["seed"])

2024-11-20 12:02:54.689274: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-20 12:02:54.717053: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


exp_016


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
set seed: 46


In [4]:
import glob
import os
import pickle
import warnings
from pathlib import Path

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


In [5]:
class Config:
    N_FOLD = 5
    RANDOM_SATE = 42


NB = "exp1015"

In [6]:
ROOT_DIR = Path("/kaggle")
DATA_DIR = ROOT_DIR / Path("input/atmaCup#18_dataset")

In [7]:
train_df = pl.read_csv(DATA_DIR / "train_features.csv")
test_df = pl.read_csv(DATA_DIR / "test_features.csv")

## 特徴量生成


In [8]:
# train_df と test_dfを結合（特徴量エンジニアリングをしやすくするため）
_all_df = pl.concat([train_df, test_df], how="diagonal")

In [9]:
agg_cols = [
    "vEgo",
    "aEgo",
    "steeringAngleDeg",
    "steeringTorque",
    "gas",
]  # 同一シーンから集計する値のカラム名

# 同一シーンから特徴量作成
exprs = []
exprs += [
    pl.col(agg_col).shift(-1).over("scene").alias(f"{agg_col}_shift-1") for agg_col in agg_cols
]  # 1ステップ前の時間の値
exprs += [
    pl.col(agg_col).shift(1).over("scene").alias(f"{agg_col}_shift1") for agg_col in agg_cols
]  # 1ステップ後の時間の値
exprs += [
    pl.col(agg_col).diff(-1).over("scene").alias(f"{agg_col}_diff-1") for agg_col in agg_cols
]  # 1ステップ前の時間の値との差分
exprs += [
    pl.col(agg_col).diff(1).over("scene").alias(f"{agg_col}_diff1") for agg_col in agg_cols
]  # 1ステップ後の時間の値との差分
exprs += [pl.col(agg_col).mean().over("scene").alias(f"{agg_col}_mean") for agg_col in agg_cols]  # 同一シーンの平均値
exprs += [pl.col(agg_col).std().over("scene").alias(f"{agg_col}_std") for agg_col in agg_cols]  # 同一シーンの標準偏差
exprs += [pl.col(agg_col).max().over("scene").alias(f"{agg_col}_max") for agg_col in agg_cols]  # 同一シーンの最大値
exprs += [pl.col(agg_col).min().over("scene").alias(f"{agg_col}_min") for agg_col in agg_cols]  # 同一シーンの最小値

_all_df = (
    _all_df.with_columns(
        # ID からシーンとデシ秒を作成
        pl.col("ID").str.split("_").list.get(0).alias("scene"),
        pl.col("ID").str.split("_").list.get(1).cast(pl.Int32).alias("decisecond"),
    )
    .sort(
        # shiftと diffが時系列順に並んでいる必要があるためシーンごとに時間軸でソート
        "scene",
        "decisecond",
    )
    .with_columns(exprs)
)

In [10]:
train_folds = pl.read_csv(CFG["dataset"]["train_fold_path"]).rename({"sceneID": "scene"})
_all_df = _all_df.join(train_folds, how="left", on="scene")
# assert train_df["fold"].null_count() == 0

In [11]:
# YOLOの検出結果
import json

yolo_paths = glob.glob("/kaggle/input/yolo-det/det/*.json")
yolo_dfs = []
for path in tqdm(yolo_paths):
    ID = os.path.basename(path).split(".")[0]
    with open(path, "r") as f:
        data = json.load(f)

    yolo_feature = {
        "ID": ID,
        "num_objects": len(data),
    }
    for bbox in data:
        if bbox["x1"] == bbox["x2"] or bbox["y1"] == bbox["y2"]:
            continue

        if bbox["cls"] != "car":
            continue

        # count
        if bbox["cls"] not in yolo_feature:
            yolo_feature[bbox["cls"]] = 0
        yolo_feature[bbox["cls"]] += 1

        # 最も横方向が中央にあるものの情報
        if f"center_x_{bbox['cls']}" not in yolo_feature:
            yolo_feature[f"center_x_{bbox['cls']}"] = -1
        current_dist = abs(yolo_feature[f"center_x_{bbox['cls']}"] - 64)
        now_center_x = (bbox["x1"] + bbox["x2"]) / 2
        now_dist = abs(now_center_x - 64)
        if now_dist < current_dist:
            yolo_feature[f"center_x_{bbox['cls']}"] = now_center_x
            yolo_feature[f"center_y_{bbox['cls']}"] = (bbox["y1"] + bbox["y2"]) / 2
            yolo_feature[f"width_{bbox['cls']}"] = bbox["x2"] - bbox["x1"]
            yolo_feature[f"height_{bbox['cls']}"] = bbox["y2"] - bbox["y1"]
            yolo_feature[f"bottom_{bbox['cls']}"] = bbox["y2"]
            yolo_feature[f"area_{bbox['cls']}"] = (bbox["x2"] - bbox["x1"]) * (bbox["y2"] - bbox["y1"])
            yolo_feature[f"aspect_ratio_{bbox['cls']}"] = (bbox["x2"] - bbox["x1"]) / (bbox["y2"] - bbox["y1"])
            yolo_feature[f"conf_{bbox['cls']}"] = bbox["conf"]
    yolo_dfs.append(yolo_feature)
yolo_df = pl.DataFrame(yolo_dfs)

_all_df = _all_df.join(yolo_df, how="left", on="ID")

100%|██████████| 45098/45098 [00:00<00:00, 79907.81it/s]


## feature and target


In [12]:
targets = [
    "x_0",
    "y_0",
    "z_0",
    "x_1",
    "y_1",
    "z_1",
    "x_2",
    "y_2",
    "z_2",
    "x_3",
    "y_3",
    "z_3",
    "x_4",
    "y_4",
    "z_4",
    "x_5",
    "y_5",
    "z_5",
]

# 使う特徴量を指定するより使わない特徴量を指定するほうが試行錯誤が楽
del_columns = targets + ["ID", "scene", "gearShifter", "fold"]

features = list(set(_all_df.columns) - set(del_columns))
features.sort()

print(features)

['aEgo', 'aEgo_diff-1', 'aEgo_diff1', 'aEgo_max', 'aEgo_mean', 'aEgo_min', 'aEgo_shift-1', 'aEgo_shift1', 'aEgo_std', 'area_car', 'aspect_ratio_car', 'bottom_car', 'brake', 'brakePressed', 'car', 'center_x_car', 'center_y_car', 'conf_car', 'decisecond', 'gas', 'gasPressed', 'gas_diff-1', 'gas_diff1', 'gas_max', 'gas_mean', 'gas_min', 'gas_shift-1', 'gas_shift1', 'gas_std', 'height_car', 'leftBlinker', 'num_objects', 'rightBlinker', 'steeringAngleDeg', 'steeringAngleDeg_diff-1', 'steeringAngleDeg_diff1', 'steeringAngleDeg_max', 'steeringAngleDeg_mean', 'steeringAngleDeg_min', 'steeringAngleDeg_shift-1', 'steeringAngleDeg_shift1', 'steeringAngleDeg_std', 'steeringTorque', 'steeringTorque_diff-1', 'steeringTorque_diff1', 'steeringTorque_max', 'steeringTorque_mean', 'steeringTorque_min', 'steeringTorque_shift-1', 'steeringTorque_shift1', 'steeringTorque_std', 'vEgo', 'vEgo_diff-1', 'vEgo_diff1', 'vEgo_max', 'vEgo_mean', 'vEgo_min', 'vEgo_shift-1', 'vEgo_shift1', 'vEgo_std', 'width_car']


In [13]:
# MAEを計算
def evaluation(true_values, pred_values):
    abs_diff = abs(true_values - pred_values)
    mae = np.mean(
        abs_diff.reshape(
            -1,
        )
    )
    return mae

### encoding


In [14]:
# label encdoding
categorical_columns = ["gearShifter"]

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    _all_df = _all_df.with_columns(pl.Series(le.fit_transform(_all_df[col])).alias(f"{col}_le"))
cate_features = [f"{col}_le" for c in categorical_columns]
features = list(set(features) | set(cate_features))

# count encoding
count_enc = ["gearShifter"]
_all_df = _all_df.with_columns([pl.col(c).count().over(c).alias(f"{c}_count") for c in count_enc])
count_features = [f"{c}_count" for c in count_enc]
features = list(set(features) | set(count_features))


train_df = train_df.join(_all_df, how="left", on="ID")
test_df = test_df.join(_all_df, how="left", on="ID")

In [15]:
def train_lgbm(target):
    params = {
        "boosting_type": "gbdt",
        "metric": "mae",  # 今回の評価指標がMAEを使用
        "objective": "regression",
        "n_jobs": -1,
        "seed": Config.RANDOM_SATE,
        "learning_rate": 0.01,
        # "device": "gpu"
        "verbosity": -1,
    }

    oof_pred = np.zeros(len(train_df))
    y_pred = np.zeros(len(test_df))
    models = []
    cv_scores = {}

    for fold in range(5):
        print(f"fold{fold}: ", end="")

        # TrainとTestに分割
        x_train = train_df.filter(pl.col("fold") != fold).select(features)
        x_val = train_df.filter(pl.col("fold") == fold).select(features)
        y_train = train_df.filter(pl.col("fold") != fold).select(target)
        y_val = train_df.filter(pl.col("fold") == fold).select(target)

        test = test_df[features]

        # create Dataset
        train_set = lgb.Dataset(
            x_train.to_pandas(),
            y_train.to_pandas(),
            categorical_feature=cate_features,
            free_raw_data=False,
        )
        val_set = lgb.Dataset(
            x_val.to_pandas(),
            y_val.to_pandas(),
            categorical_feature=cate_features,
            free_raw_data=False,
        )

        # train
        model = lgb.train(
            params,
            train_set,
            valid_sets=[train_set, val_set],
            num_boost_round=10000,
            callbacks=[
                lgb.early_stopping(stopping_rounds=100, verbose=False),
                lgb.log_evaluation(500000),
            ],
        )

        models.append(model)

        fold_pred = model.predict(x_val.to_pandas())

        score = evaluation(y_val.to_numpy().reshape(-1), fold_pred)
        cv_scores[f"cv{fold}"] = score

        # oof_pred[test_index] = fold_pred
        oof_pred[train_df["fold"].to_numpy() == fold] = fold_pred

        y_pred += model.predict(test.to_pandas()) / Config.N_FOLD

        print(f"{score}")

    oof_score = evaluation(train_df[target].to_numpy().reshape(-1), oof_pred)
    print(f"OOF score is {oof_score}")

    return oof_pred, y_pred, models

## 学習


In [16]:
# def add_dt_features(train: pl.DataFrame):
#     """dt秒後の特徴"""
#     train = train.with_columns(
#         # vt
#         (pl.col("vEgo") * pl.col("dt").cast(pl.Float32)).alias("linear_movement@dt"),
#         # vt + 0.5at^2
#         ((pl.col("vEgo") + 0.5 * pl.col("aEgo") * pl.col("dt").cast(pl.Float32) ** 2).alias("movement@dt")),
#         # v + at
#         (pl.col("vEgo") + pl.col("aEgo") * pl.col("dt").cast(pl.Float32)).alias("velocity@dt"),
#     )
#     return train


models_dict = {}
test_pred = []
oof_pred = []
for target in targets:
    print("=" * 50)
    print(f"# {target}")
    print("=" * 50)

    # # dt features
    # dt = float(target.split("_")[-1]) * 0.5 + 0.5
    # train_df = train_df.with_columns(pl.lit(dt).alias("dt"))
    # test_df = test_df.with_columns(pl.lit(dt).alias("dt"))
    # train_df = add_dt_features(train_df)
    # test_df = add_dt_features(test_df)
    # features = list(set(features) | set(["linear_movement@dt", "movement@dt", "velocity@dt"]))

    oof_preds_partial, y_pred_partial, models_partial = train_lgbm(target)
    models_dict[target] = models_partial
    oof_pred.append(oof_preds_partial)
    test_pred.append(y_pred_partial)

# x_0
fold0: 0.06166408497518744
fold1: 0.060446681161354744
fold2: 0.06338179247471751
fold3: 0.06168801271251418
fold4: 0.06254685322477438
OOF score is 0.06194547842150476
# y_0
fold0: 0.03244410596347763
fold1: 0.03181816291845839
fold2: 0.032722975611291484
fold3: 0.03200515828778085
fold4: 0.03342734766428247
OOF score is 0.03248354917959965
# z_0
fold0: 0.02586857434877633
fold1: 0.02540218413431691
fold2: 0.026390317592620685
fold3: 0.026321716653982342
fold4: 0.025640941209083987
OOF score is 0.025924745492594787
# x_1
fold0: 0.13098353152635395
fold1: 0.13087087423535929
fold2: 0.13387520059348326
fold3: 0.13085865928417176
fold4: 0.134510998967475
OOF score is 0.13221982441565297
# y_1
fold0: 0.07410883504516325
fold1: 0.0722308296426143
fold2: 0.07494224360118068
fold3: 0.07304280191404393
fold4: 0.07483372026158128
OOF score is 0.07383169248310721
# z_1
fold0: 0.053888117693342494
fold1: 0.05236267453700406
fold2: 0.05448798448988511
fold3: 0.05473681602724649
fold4: 0.053

In [17]:
evaluation(train_df[targets].to_numpy(), np.vstack(oof_pred).T)

0.21104068744171764

# submit ファイル作成


In [18]:
sub_col_names = [
    "x_0",
    "y_0",
    "z_0",
    "x_1",
    "y_1",
    "z_1",
    "x_2",
    "y_2",
    "z_2",
    "x_3",
    "y_3",
    "z_3",
    "x_4",
    "y_4",
    "z_4",
    "x_5",
    "y_5",
    "z_5",
]
sub_df = pl.DataFrame(np.vstack(test_pred).T, schema=sub_col_names)
sub_df = sub_df.with_columns(pl.Series("ID", test_df["ID"]))
sub_df.write_csv(os.path.join(CFG["output_dir"], "submission.csv"))

oof_df = pl.DataFrame(np.vstack(oof_pred).T, schema=sub_col_names)
oof_df = oof_df.with_columns(pl.Series("ID", train_df["ID"]))
oof_df.write_csv(os.path.join(CFG["output_dir"], "oof.csv"))

display(sub_df)
sub_df.drop("ID").write_csv(os.path.join(CFG["output_dir"], "submission_wo_ID.csv"))


x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,ID
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
1.455648,-0.050018,0.003142,3.028354,-0.11188,0.007276,4.526829,-0.208716,0.008452,5.862666,-0.186544,0.022955,7.265309,-0.148999,0.020651,8.618302,-0.061914,0.020857,"""012baccc145d400c896cb82065a93d…"
0.931759,0.378611,-0.000151,1.727811,0.96285,0.001481,2.356016,1.620007,0.011423,2.918974,2.393055,0.015205,3.7591,3.23484,0.017041,4.396883,4.146869,0.023659,"""012baccc145d400c896cb82065a93d…"
1.580343,0.012554,0.00411,3.263016,0.011234,0.006631,4.809744,0.036665,0.012191,6.294947,0.088492,0.010007,7.761515,0.122175,-0.000794,9.052538,0.184131,0.0046,"""012baccc145d400c896cb82065a93d…"
0.832457,0.067855,-0.001345,1.645893,0.241727,-0.004596,2.379354,0.588941,-0.007969,2.951857,0.947877,-0.013307,3.557095,1.730979,-0.020588,4.199789,2.647696,-0.025641,"""012baccc145d400c896cb82065a93d…"
0.819643,0.003443,-0.001796,1.421269,0.003446,-0.006486,1.889901,0.012635,-0.020056,2.240321,0.011022,-0.036228,2.193091,0.000391,-0.054343,1.721956,-0.009503,-0.070152,"""01d738e799d260a10f6324f78023b3…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
6.537419,0.008363,0.018345,13.800382,0.047219,0.046983,21.009712,0.125803,0.079308,28.269568,0.203972,0.091345,35.549006,0.310601,0.100467,42.993585,0.444018,0.100446,"""ff4f00a76fbf4db0cb15579c7c6086…"
6.997276,0.000465,0.005644,14.877982,-0.014297,0.012522,22.910983,-0.038783,0.020608,31.034881,-0.087646,0.035036,39.19934,-0.117717,0.040591,47.447055,-0.158336,0.047015,"""ff4f00a76fbf4db0cb15579c7c6086…"
7.413496,-0.001997,0.007757,15.659215,-0.022804,0.017406,23.871527,-0.068412,0.033213,32.024362,-0.142547,0.067403,40.06737,-0.212788,0.089625,47.998011,-0.283517,0.137738,"""ff4f00a76fbf4db0cb15579c7c6086…"
